# Part I. ETL Pipeline for Pre-Processing the Files

## THE FOLLOWING CODE IS USED FOR PRE-PROCESSING EVENT FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new_orig.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. NoSQL Database Project with Apache Cassandra. 

 The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is completed:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Connect to a Cassandra instance on local machine 
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# Instantiate session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create Cassandra KEYSPACE
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class': 'SimpleStrategy', 'replication_factor' :1}"""
)
    print("KEYSPACE CREATED")
except Exception as e:
    print(e)

KEYSPACE CREATED


#### Set Keyspace

In [7]:
#Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
    print("KEYSPACE SET")
except Exception as e:
    print(e)

KEYSPACE SET


## Queries created below provide insight into the following data questions. 

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
##  Create Schema and table that will provide artist, song title, and song's length in the music app history heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS artist_sessions"
query = query + "(sessionID int, itemInSession int, artist text, song text, length double,  PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
# Open and iterate through event csv file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Create insert query
        query = "INSERT INTO artist_sessions(sessionID, itemInSession, artist, song, length )"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign column elements for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]),line[0], line[9], int(float(line[5])) ))

#### Query below extracts artist, song, and length based on a provided sessionId and itemInSession. 
#### Since the query is based on the sessionId and itemInSesson, the composite key is comprised of those columns.  The optimal approach to partioning the data is to partion the data based on the sessionId, and then cluster/sort based on itemInSession values for more efficent querying of the data.  

In [11]:
# Execute select statement used to model table.
query = "select artist, song, length from artist_sessions where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [12]:
#Iterate and print out query results.
#for row in rows:
#    print(row.artist,row.song, row.length)

In [13]:
#Create pandas DataFrame for results
d = []

for row in rows:
    d.append((row.artist,row.song, row.length)) 

d = pd.DataFrame(d, columns = ['artist','song','length'])
print(d)

      artist                             song  length
0  Faithless  Music Matters (Mark Knight Dub)   495.0


In [14]:
## Create Schema and table that will provide name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS user_library"
query = query + "(userId int, sessionId int, artist text, song text, firstName text, lastName text, \
itemInSession int, PRIMARY KEY((userId, sessionId), song, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Open and iterate through event csv file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Create insert query
        query = "INSERT INTO user_library(userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign column elements for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]),line[0], line[9], line[1], line[4], int(line[3])))
                    


#### Query below extracts artist, song, and userId based on a given userId and sessionId. 
#### Since the query is based on the userId and sessionId, the composite key includes those columns. the song and itemInSession data are also included in the composite key, so that song data can be sorted by the itemInSession values.  The optimal approach to partioning the data is to partion the data based on the userId, and then cluster/sort based on sessionId values for more efficent querying of the data.  

In [15]:
# Execute select statement used to model table.
query = "select artist, song, userId from user_library where userId=10 and sessionId =182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    


In [16]:
#Iterate and print out query results.
#for row in rows:
#    print(row.artist,row.song, row.userid)

In [17]:
#Create pandas DataFrame for results
c = []

for row in rows:
    c.append((row.artist,row.song, row.userid)) 

c = pd.DataFrame(c, columns = ['artist','song','userId'])
print(c)

              artist                                               song  \
0      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   
1       Three Drives                                        Greece 2000   
2   Down To The Bone                                 Keep On Keepin' On   
3  Sebastien Tellier                                          Kilometer   

   userId  
0      10  
1      10  
2      10  
3      10  


In [18]:
## Create Schema and table that will provide every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS song_user_assoc"
query = query + "(song text, userId int, artist text, firstName text, lastName text, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [19]:
# Open and iterate through event csv file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Create insert query
        query = "INSERT INTO song_user_assoc(song,  userId, artist, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign column elements for each column in the INSERT statement.
        session.execute(query, (line[9],  int(line[10]), line[0], line[1], line[4]))

#### Query below extracts user first and last name based on a song they have played. 
#### Since the query is based on the song, the composite key is comprised of  song and userId so that the data can be be partitoned and sorted efficiently.  For this data the song is he partion key, and the data is then clustered/sorted based on userId.  

In [20]:
# Execute select statement used to model table.
query = "select firstName, lastName from song_user_assoc where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)



In [21]:
#Iterate and print out query results.
#for row in rows:
#    print(row)
#    print(row.firstName, row.lastName)

In [22]:
#Create pandas DataFrame for results
e = []

for row in rows:
    e.append((row.firstname,row.lastname)) 

e = pd.DataFrame(e, columns = ['firstName','lastName'])
print(e)

    firstName lastName
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


In [23]:
## Drop tables prior to closing out sessions
query = "drop table artist_sessions"
try:
    rows = session.execute(query)
    print("artist_sessions table dropped")
except Exception as e:
    print(e)

query = "drop table user_library"
try:
    rows = session.execute(query)
    print("user_library table dropped")
except Exception as e:
    print(e)
    
query = "drop table song_user_assoc"
try:
    rows = session.execute(query)
    print("song_user_assoc table dropped")
except Exception as e:
    print(e)

artist_sessions table dropped
user_library table dropped
song_user_assoc table dropped


In [24]:
#Shutdown Sessions
session.shutdown()
cluster.shutdown()